In [1]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
import math
import json
from pandas.io.json import json_normalize

In [14]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

visits = pd.read_sql('SELECT * FROM Visits', con=db_connection)
visits['created'] = pd.to_datetime(visits['created'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['lastmodified'] = pd.to_datetime(visits['lastmodified'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['registered'] = visits['registered'].replace(0, np.nan)
visits['registered'] = pd.to_datetime(visits['registered'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['deleted'] = visits['deleted'].replace(0, np.nan)
visits['deleted'] = pd.to_datetime(visits['deleted'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['paired'] = visits['paired'].replace(0, np.nan)
visits['paired'] = pd.to_datetime(visits['paired'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['first_accessed'] = pd.to_datetime(visits['first_accessed'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['created_ip'].replace(0, np.nan, inplace=True)
visits['registered_ip'].replace(0, np.nan, inplace=True)
visits['paired_ip'].replace(0, np.nan, inplace=True)
visits['exported'].replace(0, np.nan, inplace=True)
visits['title'].fillna(value=np.nan, inplace=True)
visits['note'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)
pd.set_option("max_colwidth", 1000)

## ticket column

In [5]:
ticket_parse = visits['ticket']

def parse(string):
    if string is None:
        return {}
    else:
        try:
            return json.loads(string)
        except ValueError:
            print(string)
            return {}
        
    
json_str = ticket_parse.map(parse)

In [6]:
visits['ticket_id'] = json_str.map(lambda row: row.get('ticket_id'))

In [7]:
visits

,id,shortcode,user_id,created,lastmodified,registered,expires,paired,ticket,deleted,created_ip,registered_ip,paired_ip,processed,purchased_by_user_id,note,exported,first_accessed,ignore_first_accessed,title,ticket_id
0,794965,20141029u305,0,2014-10-29 17:10:47-04:00,2016-02-23 17:57:37-05:00,2014-10-29 17:10:47-04:00,0,2014-10-29 17:13:56-04:00,"{""tessitura_spoofing"":1,""ticket_id"":""794965"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T17:10:47-04:00""}",2014-10-29 19:19:52-04:00,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,794965
1,795317,foo001,0,2014-10-29 19:32:29-04:00,2016-02-23 17:57:37-05:00,2014-10-29 19:32:29-04:00,0,2014-10-29 19:33:06-04:00,"{""tessitura_spoofing"":1,""ticket_id"":""795317"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T19:32:29-04:00""}",2014-10-29 19:35:18-04:00,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,795317
2,795341,foo002,0,2014-10-29 19:40:51-04:00,2016-02-23 17:57:37-05:00,2014-10-29 19:40:51-04:00,0,2014-10-29 19:41:08-04:00,"{""tessitura_spoofing"":1,""ticket_id"":""795341"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T19:40:51-04:00""}",2014-10-29 19:41:21-04:00,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,795341
3,795351,foo003,0,2014-10-29 19:43:31-04:00,2016-02-23 17:57:37-05:00,2014-10-29 19:43:31-04:00,0,2014-10-29 19:44:28-04:00,"{""tessitura_spoofing"":1,""ticket_id"":""795351"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T19:43:31-04:00""}",2014-10-29 19:44:34-04:00,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,795351
4,795405,foo004,0,2014-10-29 20:13:47-04:00,2016-02-23 17:57:37-05:00,2014-10-29 20:13:47-04:00,0,2014-10-29 20:14:13-04:00,"{""tessitura_spoofing"":1,""ticket_id"":""795405"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T20:13:47-04:00""}",2014-10-29 20:14:41-04:00,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,795405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716277,132806719,56bs6,0,2020-07-09 09:11:50-04:00,2020-07-09 09:22:11-04:00,2020-07-09 09:12:12-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 940560"",""ticket_id"":""940560"",""shortcode"":""56bs6"",""performance_id"":""4254"",""performance_date"":""2020-07-09T10:00:00-04:00"",""date_attended"":""Jul 9 2020 9:12AM"",""attended"":""1"",""type"":null}",NaT,1.198075e+09,1.198075e+09,NaN,0,0,NaN,NaN,1.594301e+09,NaN,NaN,940560
716278,132806727,cvszd,0,2020-07-09 09:16:19-04:00,2020-07-09 09:22:42-04:00,2020-07-09 09:16:32-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 940561"",""ticket_id"":""940561"",""shortcode"":""cvszd"",""performance_id"":""4254"",""performance_date"":""2020-07-09T10:00:00-04:00"",""date_attended"":""Jul 9 2020 9:16AM"",""attended"":""1"",""type"":null}",NaT,1.198075e+09,1.198075e+09,NaN,0,0,NaN,NaN,1.594301e+09,NaN,NaN,940561
716279,132806735,zrbqg,0,2020-07-09 09:19:41-04:00,2020-07-09 09:22:54-04:00,2020-07-09 09:19:58-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 940562"",""ticket_id"":""940562"",""shortcode"":""zrbqg"",""performance_id"":""4254"",""performance_date"":""2020-07-09T10:00:00-04:00"",""date_attended"":""Jul 9 2020 9:19AM"",""attended"":""1"",""type"":null}",NaT,1.198075e+09,1.198075e+09,NaN,0,0,NaN,NaN,1.594301e+09,NaN,NaN,940562
716280,132806749,Jr656,0,2020-07-09 09:40:43-04:00,2020-07-09 09:40:43-04:00,2020-07-09 09:40:42-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 940563"",""ticket_id"":""940563"",""shortcode"":""Jr656"",""performance_id"":""4254"",""performance_date"":""2020-07-09T10:00:00-04:00"",""date_attended"":""Jul 9 2020 9:40AM"",""attended"":""1"",""type"":null}",NaT,1.198075e+09,1.198075e+09,NaN,0,0,NaN,NaN,NaN,NaN,NaN,940563


## first_access

In [17]:
visits.set_index('created', inplace=True)
visits_openinghours = visits.between_time('10:00', '18:00')
visits.reset_index(inplace=True)
visits_openinghours.reset_index(inplace=True)

In [26]:
def average_time_first_access():
    visit_accessed = visits_openinghours[visits_openinghours['ignore_first_accessed'].isna()]
    visit_accessed['time_between'] = visit_accessed['first_accessed'] - visit_accessed['created']
    return visit_accessed['time_between'].mean()

In [27]:
average_time_first_access()

/Users/shirin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Timedelta('51 days 14:43:34.945306')

In [28]:
def percentage_first_access():
    visit_accessed = visits_openinghours[visits_openinghours['ignore_first_accessed'].isna()]
    visit_accessed_notna = visit_accessed[visit_accessed['first_accessed'].notna()]
    return visit_accessed_notna['first_accessed'].count() / visits_openinghours['created'].count() * 100

In [29]:
percentage_first_access()

16.059409139429032